### Australian Freight Export Data Cleaning
##### Data Cleaning and Preprocessing for 2024-2025 Export Data

This notebook focuses on cleaning and preprocessing the ABS export data before analysis.


In [ ]:
# Install PySpark (run this first!)
%pip install pyspark

# Check if Java is installed (required for Spark)
!java -version

# Check Python packages
!pip list | grep spark


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#set display options for better viewing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)


In [ ]:
# Check original product descriptions BEFORE any processing


#### 1. Data Loading


In [ ]:
# Load the export data
df = pd.read_csv('data/exports_2024_2025.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")



In [ ]:
df.head(3)

#### 2. Initial Data Exploration


In [ ]:
# Basic information about the dataset
print("=== DATASET OVERVIEW ===")
print(f"Total records: {len(df):,}")
print(f"Total columns: {len(df.columns)}")

print("\n=== COLUMN INFORMATION ===")
print(df.info())


In [ ]:
# Check for missing values
print("=== MISSING VALUES ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

# Check if any missing values exist
if missing_data.sum() == 0:
    print("No missing values found!")
else:
    print(missing_data)
    print(missing_percent)

#### 3. Data Cleaning and Conversion


In [ ]:
# Convert numeric columns and handle missing values
print("=== CLEANING DATA ===")

# Rename columns for clarity
df = df.rename(columns={
    'sitc': 'product_description',
    'sitc_code': 'prod_descpt_code'
})
print("Renamed 'sitc' to 'product_description' and 'sitc_code' to 'prod_descpt_code'")

# Convert numeric columns
numeric_columns = ['quantity', 'gross_weight_tonnes', 'value_fob_aud']
for col in numeric_columns:
    if col in df.columns:
        print(f"Converting {col} to numeric...")
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Fill missing numeric values with 0
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            print(f"  Filling {missing_count:,} missing values with 0")
            df[col] = df[col].fillna(0)
        
        # Convert negative values to 0
        negative_count = (df[col] < 0).sum()
        if negative_count > 0:
            print(f"  Converting {negative_count:,} negative values to 0")
            df[col] = df[col].clip(lower=0)

# Fill missing text values
text_columns = ['country_of_destination', 'product_description']
for col in text_columns:
    if col in df.columns:
        missing_count = df[col].isnull().sum()
        if missing_count > 0:
            print(f"Filling {missing_count:,} missing values in {col} with 'Unknown'")
            df[col] = df[col].fillna('Unknown')

print("Data cleaning completed!")


In [ ]:
# Inspect product descriptions
print("=== SAMPLE PRODUCT DESCRIPTIONS ===")
print(df['product_description'].head(10))
print(f"\n=== STATISTICS ===")
print(f"Unique products: {df['product_description'].nunique()}")
print(f"Max length: {df['product_description'].str.len().max()}")
print(f"Min length: {df['product_description'].str.len().min()}")

# Check for whitespace issues (define pattern outside f-string to avoid backslash error)
extra_whitespace_count = df['product_description'].str.contains(r'\s{2,}', regex=True).sum()
trailing_space_count = (df['product_description'] != df['product_description'].str.strip()).sum()

print(f"\nAny with extra whitespace: {extra_whitespace_count}")
print(f"Any with leading/trailing spaces: {trailing_space_count}")

df.head(3)

In [ ]:
# Clean product descriptions
print("\n=== CLEANING PRODUCT DESCRIPTIONS ===")

# Strip leading/trailing whitespace
df['product_description'] = df['product_description'].str.strip()

# Replace multiple spaces with single space
df['product_description'] = df['product_description'].str.replace(r'\s+', ' ', regex=True)

# Remove line breaks and tabs
df['product_description'] = df['product_description'].str.replace(r'[\n\r\t]+', ' ', regex=True)

# Standardize quotation marks
df['product_description'] = df['product_description'].str.replace('"', '', regex=False)
df['product_description'] = df['product_description'].str.replace("'", '', regex=False)

# Optional: Title case for consistency (comment out if you prefer original casing)
# df['product_description'] = df['product_description'].str.title()

print(f"Product descriptions cleaned!")
print(f"Unique products: {df['product_description'].nunique()}")


In [ ]:
# Standardization section removed - now handled by SITC mapping
print("✅ Standardization removed - SITC mapping will handle unclassified products")


In [ ]:

# Check for whitespace issues (define pattern outside f-string to avoid backslash error)
extra_whitespace_count = df['product_description'].str.contains(r'\s{2,}', regex=True).sum()
trailing_space_count = (df['product_description'] != df['product_description'].str.strip()).sum()

print(f"\nAny with extra whitespace: {extra_whitespace_count}")
print(f"Any with leading/trailing spaces: {trailing_space_count}")

df.head(3)

In [ ]:
#### 3.5 Derived Features & Transformations


In [ ]:
# This section creates derived features before final duplicate removal

In [ ]:
# Extract year and month into separate columns
print("\n=== CREATING DERIVED FEATURES ===")

# The original month column format is "Month_Name YYYY" (e.g., "August 2024")
# Extract year (4 digits at the end)
df['year'] = df['month'].str.extract(r'(\d{4})')[0]
df['year'] = pd.to_numeric(df['year'], errors='coerce')

# Extract month name (everything before the year) - get first column as Series
month_name_temp = df['month'].str.extract(r'^([A-Za-z]+)')[0]

# Convert month name to month number
month_to_number = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df['month_number'] = month_name_temp.map(month_to_number)

# Replace the original month column with just the month name
df['month'] = month_name_temp

# Create value per tonne metric
df['value_per_tonne'] = df['value_fob_aud'] / df['gross_weight_tonnes'].replace(0, np.nan)
df['data_processed_date'] = datetime.now().strftime('%Y-%m-%d')

print(f"Years found: {sorted(df['year'].dropna().unique())}")
print(f"Months found: {sorted(df['month_number'].dropna().unique())}")
print(f"Month names: {df['month'].unique()}")
print(f"Value per tonne calculated for {df['value_per_tonne'].notna().sum():,} records")


In [ ]:
# Final duplicate check and removal (after all transformations)
print("\n=== FINAL DUPLICATE CHECK ===")
duplicates = df.duplicated().sum()
print(f"Found {duplicates:,} duplicate rows")

if duplicates > 0:
    # Show some duplicate rows as examples
    print("\nExample of duplicate rows:")
    duplicate_mask = df.duplicated(keep=False)
    sample_duplicates = df[duplicate_mask].head(4).sort_values(by=['month', 'product_description'])
    print(sample_duplicates[['month', 'year', 'product_description', 'country_of_destination', 
                              'value_fob_aud', 'gross_weight_tonnes']].to_string())
    
    # Remove duplicates
    print(f"\nRemoving {duplicates:,} duplicate rows...")
    df = df.drop_duplicates()
    print(f"✅ Duplicates removed! New shape: {df.shape}")
else:
    print("✅ No duplicates found!")


In [ ]:
df.head(5)

#### 4. Save Cleaned Data


In [ ]:
# SITC CODE MAPPING TO PRODUCT DESCRIPTIONS
print("=== SITC CODE MAPPING ===")

# Import SITC mapping from external file
from sitc_mapping import map_sitc_to_product, get_unclassified_patterns

# Apply SITC mapping to unclassified products
print("Mapping SITC codes to product descriptions...")

# Count unclassified products before mapping
unclassified_patterns = '|'.join(get_unclassified_patterns())
unclassified_before = df['product_description'].str.contains(unclassified_patterns, case=False, na=False).sum()
print(f"Unclassified products before mapping: {unclassified_before:,}")

# Create new product description based on SITC codes for unclassified products
mask_unclassified = df['product_description'].str.contains(unclassified_patterns, case=False, na=False)
df.loc[mask_unclassified, 'product_description'] = df.loc[mask_unclassified, 'prod_descpt_code'].apply(map_sitc_to_product)

# Count unclassified products after mapping
unclassified_after = df['product_description'].str.contains(unclassified_patterns, case=False, na=False).sum()
print(f"Unclassified products after mapping: {unclassified_after:,}")
print(f"Products successfully mapped: {unclassified_before - unclassified_after:,}")

print("SITC code mapping completed!")


In [ ]:
# COUNTRY CODE MAPPING TO COUNTRY NAMES
print("\n=== COUNTRY CODE MAPPING ===")

# Import country mapping from external file
from country_mapping import map_country_code_to_name, get_problematic_country_patterns

# First, let's check what values are actually in the country_of_destination column
print("Checking country_of_destination column values...")
print("\nUnique values in country_of_destination column:")
country_values = df['country_of_destination'].value_counts()
print(f"Total unique countries: {len(country_values)}")
print("\nTop 20 countries:")
print(country_values.head(20))

print("\nChecking for problematic entries...")
problematic_patterns = get_problematic_country_patterns()
problematic_countries = []

for pattern in problematic_patterns:
    matches = df['country_of_destination'].str.contains(pattern, case=False, na=False)
    if matches.any():
        problematic_countries.extend(df[matches]['country_of_destination'].unique())

if problematic_countries:
    print(f"\nFound problematic country entries:")
    for country in set(problematic_countries):
        count = (df['country_of_destination'] == country).sum()
        print(f"  '{country}': {count:,} records")
else:
    print("\nNo obvious problematic entries found.")

# Apply country code mapping ONLY to truly problematic country names
print("\nMapping country codes to country names...")

# Import the new function
from country_mapping import is_problematic_country_name

# Count truly problematic entries before mapping
problematic_mask = df['country_of_destination'].apply(is_problematic_country_name)
missing_countries_before = problematic_mask.sum()
print(f"Truly problematic country details before mapping: {missing_countries_before:,}")

# Only map entries that are actually problematic
if missing_countries_before > 0:
    df.loc[problematic_mask, 'country_of_destination'] = df.loc[problematic_mask, 'country_of_destination_code'].apply(map_country_code_to_name)

# Count problematic entries after mapping
problematic_mask_after = df['country_of_destination'].apply(is_problematic_country_name)
missing_countries_after = problematic_mask_after.sum()
print(f"Truly problematic country details after mapping: {missing_countries_after:,}")
print(f"Countries successfully mapped: {missing_countries_before - missing_countries_after:,}")

# Show examples of what was mapped
if missing_countries_before > missing_countries_after:
    print(f"\nExamples of successful mappings:")
    mapped_entries = df[problematic_mask & ~problematic_mask_after].head(5)
    for _, row in mapped_entries.iterrows():
        print(f"  Code: {row['country_of_destination_code']} → Country: {row['country_of_destination']}")

print(" Smart country code mapping completed!")

# Show some examples of the mappings
print("\n=== MAPPING EXAMPLES ===")
print("Sample SITC code mappings:")
sample_sitc = df[df['prod_descpt_code'].notna()]['prod_descpt_code'].head(5)
for code in sample_sitc:
    mapped = map_sitc_to_product(code)
    print(f"  {code} → {mapped}")

print("\nSample country code mappings:")
sample_countries = df[df['country_of_destination_code'].notna()]['country_of_destination_code'].head(5)
for code in sample_countries:
    mapped = map_country_code_to_name(code)
    print(f"  {code} → {mapped}")


In [ ]:
# Save the cleaned dataset with SITC and country mappings
import os

output_file = 'data/exports_cleaned.csv'
df.to_csv(output_file, index=False)

print(f" Cleaned data saved to: {output_file}")
print(f"Final dataset shape: {df.shape}")
print(f"File size: {os.path.getsize(output_file) / 1024**2:.2f} MB")

# Final Summary
print(f"\n DATA CLEANING COMPLETED WITH MAPPINGS!")
print("=" * 50)
print(f" Records: {len(df):,}")
print(f" Countries: {df['country_of_destination'].nunique()}")
print(f" Products: {df['product_description'].nunique()}")
print(f" Total Value: AUD ${df['value_fob_aud'].sum():,.2f}")
print(f"Total Weight: {df['gross_weight_tonnes'].sum():,.2f} tonnes")



In [ ]:
# Check what products have SITC code 98888
print("=== PRODUCTS WITH SITC CODE 98888 ===")

# Look at the original product descriptions for records with SITC code 98888
sitc_98888_records = df[df['prod_descpt_code'] == 98888]

if len(sitc_98888_records) > 0:
    print(f"Found {len(sitc_98888_records):,} records with SITC code 98888")
    print("\nOriginal product descriptions:")
    original_descriptions = sitc_98888_records['product_description'].value_counts()
    print(original_descriptions)
    
    print(f"\nTotal value for SITC 98888: ${sitc_98888_records['value_fob_aud'].sum()/1e9:.2f}B")
    print(f"Total weight for SITC 98888: {sitc_98888_records['gross_weight_tonnes'].sum()/1e6:.2f}M tonnes")
    
    print("\nTop 5 countries for SITC 98888:")
    country_distribution = sitc_98888_records.groupby('country_of_destination')['value_fob_aud'].sum().sort_values(ascending=False).head()
    for country, value in country_distribution.items():
        print(f"  {country}: ${value/1e9:.2f}B")
else:
    print("No records found with SITC code 98888")

In [ ]:
# FIX COUNTRY CODE ISSUES
print("=== FIXING COUNTRY CODE ISSUES ===")

# Check for "Country Code XXX" entries
country_code_entries = df[df['country_of_destination'].str.contains('Country Code', na=False)]
print(f"Found {len(country_code_entries):,} records with 'Country Code XXX' entries")

if len(country_code_entries) > 0:
    print("\nTop 10 Country Code entries:")
    country_code_counts = country_code_entries['country_of_destination'].value_counts().head(10)
    for country_code, count in country_code_counts.items():
        print(f"  {country_code}: {count:,} records")
    
    # Fix the country codes by extracting the code and mapping it properly
    print("\nFixing Country Code entries...")
    mask = df['country_of_destination'].str.contains('Country Code', na=False)
    
    # Extract the country code (remove "Country Code " prefix)
    df.loc[mask, 'country_of_destination'] = df.loc[mask, 'country_of_destination'].str.replace('Country Code ', '')
    
    # Now apply proper mapping using the updated function
    df.loc[mask, 'country_of_destination'] = df.loc[mask, 'country_of_destination'].apply(map_country_code_to_name)
    
    print(" Country Code entries fixed!")
    
    # Verify the fix
    remaining_country_codes = df[df['country_of_destination'].str.contains('Country Code', na=False)]
    print(f"Remaining 'Country Code XXX' entries: {len(remaining_country_codes):,}")
    
    if len(remaining_country_codes) == 0:
        print("All Country Code entries have been successfully mapped!")
    else:
        print("\nRemaining entries:")
        for country_code, count in remaining_country_codes['country_of_destination'].value_counts().items():
            print(f"  {country_code}: {count:,} records")
else:
    print("No 'Country Code XXX' entries found!")

print("\nCountry code fixing completed!")


In [ ]:
# SAVE CLEANED DATA WITH FIXED COUNTRY CODES
print("=== SAVING CLEANED DATA ===")

# Save the cleaned data with fixed country codes
output_file = 'data/exports_cleaned.csv'
df.to_csv(output_file, index=False)

print(f"Cleaned data saved to: {output_file}")
print(f"Total records saved: {len(df):,}")
print(f"Total unique countries: {df['country_of_destination'].nunique():,}")

